In [15]:
from crewai import Agent, Task, LLM , Crew
import os
import agentops

from dotenv import load_dotenv
from fpdf import FPDF
from PyPDF2 import PdfReader
import re




In [16]:
load_dotenv()  # reads .env into environment

groq_key = os.getenv("GROQ_API_KEY")
agentops_key = os.getenv("AGENTOPS_API_KEY")

In [17]:

# agentops.init(api_key=agentops_key, auto_start_session=True)


In [18]:
outputDir = "./output"
os.makedirs(outputDir, exist_ok=True)

In [19]:
basic_llm = LLM(model="groq/llama-3.1-8b-instant", temperature=0.7, api_key=groq_key)

## Summarizer Agent

In [20]:
def read_pdf(file_path):
    reader = PdfReader(file_path)
    return "\n".join(page.extract_text() for page in reader.pages if page.extract_text())

pdf_text = read_pdf("input/sample.pdf")


In [21]:
summarizer_agent = Agent(
    role = "Summarizer Agent",
    goal = "To provide a concise summary of any given document or text, ensuring clarity, accuracy, and output in a user-friendly PDF format.",
    
    backstory = "\n".join([
            "The agent is designed to assist users in quickly understanding the main points of lengthy documents or articles.", 
            "It should distill complex information into easily digestible summaries while maintaining the original context and meaning.", 
            "The agent can also summarize weekly thoughts, reflections, and tasks to support productivity.",
            "The agent should generate a PDF file containing the summary and save it to the output directory.",
            "The agent should support multiple languages and be able to summarize content in the user's preferred language.",
            "The agent should handle various document formats, including PDFs, articles, and plain text.",
    ]),
    
    llm = basic_llm,
    verbose=True
    )


summarizer_task = Task(
    description =  f"""
        Summarize the following PDF text:{pdf_text}

        Read through the document carefully and identify the key points, main ideas, and essential information.
        Create a concise summary that captures the essence of the original text while omitting unnecessary details.
        Ensure that the summary is clear, coherent, and easy to understand.
        Generate a PDF file containing the summary and save it to the output directory.
        """,
        
    expected_output= "\n".join([
        "A well-structured summary containing a concise and accurate summary of the provided document in PDF format.",
        "The summary should be saved as a PDF file in the output directory.",
        "The summary should be clear and concise summary, formatted in short paragraphs or bullet points for easy reading."
        ]),
    agent=summarizer_agent,
    )

output_file_path = os.path.join(outputDir, "summary.pdf")

## Timetable Agent

In [22]:
timetable_agent = Agent(
    role = "Timetable Agent",
    goal = "To create a detailed weekly timetable based on user-provided tasks and priorities, ensuring optimal time management and productivity.",
    backstory = """"
            The agent is designed to assist users in organizing their weekly schedule by allocating time slots for various tasks and activities., 
            It should consider task priorities, deadlines, and estimated durations to create an efficient timetable.,
            The agent should generate a PDF file containing the timetable and save it to the output directory.,
            The agent should support multiple languages and be able to create timetables in the user's preferred language.,
            The agent should handle various input formats, including plain text, lists, and structured data (e.g., JSON).,
    """,
    llm = basic_llm,
    verbose=True
    )

timetable_task = Task(
    description= """
        Create a weekly timetable based on the following tasks and priorities: {schedule_text}
        Analyze each task and its priority level.
        Distribute the tasks across the week while ensuring an efficient workload balance.
        Include short breaks, rest periods, and buffer time between major tasks.
        Ensure the timetable is clear, easy to read, and organized by day and hour.
        
        Output MUST be in the following structured format (no extra text):
        Day | Time Slot | Task | Duration | Priority
        Monday | 09:00 - 11:00 | Complete project report | 2 hours | High
        Monday | 11:00 - 12:00 | Team meeting | 1 hour | Medium
        ...
    """,
    
    expected_output= """
        A well-structured weekly timetable in PDF format, saved in the output directory.
        The timetable should clearly display each day of the week with assigned time slots for all tasks.
        It should include task names, durations, priorities, and any free or break periods.
        The final PDF should be saved in the output directory as 'timetable.pdf'.  
        
        A clean, consistent table text with five columns separated by '|':
        Day | Time Slot | Task | Duration | Priority.
        No explanations or comments.
          
        """,
    agent=timetable_agent,
    )

output_file_path = os.path.join(outputDir, "timetable.pdf")

## Running the Agent

In [23]:
multi_agents = Crew(
    agents=[summarizer_agent,
            timetable_agent
            ],
    tasks=[summarizer_task,
           timetable_task
           ],
)

In [24]:
result = multi_agents.kickoff({
    "task": "Summarize this PDF",
    "input": pdf_text,
    
    
    "schedule_text": """
    
    - Task 1: Complete project report (High Priority, 2 hours)
    - Task 2: Team meeting (Medium Priority, 1 hour)
    - Task 3: Respond to emails (Low Priority, 30 minutes)""",
    
    "output_dir": "output/",
})

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Summarizer Agent                                                                                        │
│                                                                                                                 │
│  Task:                                                                                                          │
│          Summarize the following PDF text:                                                                      │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│  DEBI Robot ics Challenge  2023                                                                                 │
│  Rules and Guidelines                                                                                           │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│  Sponsored by:       In Cooperation with:                                                                       │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│  2/4                                                                                                            │
│                                                                                                                 │
│  Challenge  Description                                                                                         │
│  In continuation of the efforts of Digital Egypt Builders Imitative (DEBI)1, launched by the Egyptian Ministry  │
│  of                                                                                                             │
│  Communications and Information Technology , the DEBI’s Robotics & IoT Track  launches the annual               │
│  DEBI Robot ics Challenge . Inspired by the GdR TurtleB

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Summarizer Agent                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  DEBI Robotics Challenge 2023 Rules and Guidelines                                                              │
│                                                                                                                 │
│  Sponsored by: In Cooperation with:                                                                             │
│                                                                                                                 │
│  Challenge Description                                                                                          │
│  The DEBI Robotics Challenge 2023 is a physical competition to advance robotic software and autonomous          │
│  capabilities. Inspired by the GdR TurtleBot Challenge 2018, the challenge is a one-vs-one match between        │
│  robots in a predefined playground. The playground is divided into two identical halves, with three randomly    │
│  placed balls on each side. The main mission is to reduce the number of balls in each team's area by moving     │
│  them to the opponent's side.                                                                                   │
│                                                                                                                 │
│  Rules and Regulations                                                                                          │
│  1. All team members must be physically present on the competition day.                                         │
│  2. The competition is based on knockout matches, and DEBI reserves the right to change the competition style   │
│  without prior notice.                                                                                          │
│  3. The robot must be fully autonomous and cannot cross the red line or the playground walls.                   │
│  4. If a robot crosses the boundaries, it must start over from the initial position, and any moved balls will   │
│  be returned to their original side.                                                                            │
│  5. The match duration is 10 minutes, and if no team wins, the match will resume for another 3 minutes.         │
│  6. If the match ends in a draw, a golden-ball scheme will be followed, and the first team that moves a ball    │
│  to the other side will win.                                                                                    │
│  7. Technical Specifications                                                                                    │
│  - DEBI provides TurtleBot 3 Waffle Pi equipped with OpenMANIPULATOR-X with installed configuration of Ubuntu   │
│  20.04 and ROS1 Noetic Ninjemys.                                                                                │
│  - Playground dimensions: 420 cm x 300 cm, walls height is 31 cm.                                               │
│  - Balls: diameter = 5.5 cm, Color = random.                                                                    │
│                                                                                                                 │
│  Rewards                                                                                                        │
│  The winning teams will receive a financial award, as follows:                                                  │
│  - First place: 80,000 EGP                             

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Timetable Agent                                                                                         │
│                                                                                                                 │
│  Task:                                                                                                          │
│          Create a weekly timetable based on the following tasks and priorities:                                 │
│                                                                                                                 │
│      - Task 1: Complete project report (High Priority, 2 hours)                                                 │
│      - Task 2: Team meeting (Medium Priority, 1 hour)                                                           │
│      - Task 3: Respond to emails (Low Priority, 30 minutes)                                                     │
│          Analyze each task and its priority level.                                                              │
│          Distribute the tasks across the week while ensuring an efficient workload balance.                     │
│          Include short breaks, rest periods, and buffer time between major tasks.                               │
│          Ensure the timetable is clear, easy to read, and organized by day and hour.                            │
│                                                                                                                 │
│          Output MUST be in the following structured format (no extra text):                                     │
│          Day | Time Slot | Task | Duration | Priority                                                           │
│          Monday | 09:00 - 11:00 | Complete project report | 2 hours | High                                      │
│          Monday | 11:00 - 12:00 | Team meeting | 1 hour | Medium                                                │
│          ...                                                                                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Timetable Agent                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Day | Time Slot | Task | Duration | Priority                                                                   │
│  Monday | 09:00 - 11:00 | Complete project report | 2 hours | High                                              │
│  Monday | 11:00 - 12:00 | Team meeting | 1 hour | Medium                                                        │
│  Monday | 12:00 - 12:30 | Break | 30 minutes | -                                                                │
│  Monday | 12:30 - 13:30 | Respond to emails | 1 hour | Low                                                      │
│  Monday | 13:30 - 14:00 | Break | 30 minutes | -                                                                │
│  Monday | 14:00 - 16:00 | Break and rest | 2 hours | -                                                          │
│                                                                                                                 │
│  Tuesday | 09:00 - 11:00 | Task 1: Complete project report | 2 hours | High                                     │
│  Tuesday | 11:00 - 12:00 | Break | 1 hour | -                                                                   │
│  Tuesday | 12:00 - 13:00 | Task 2: Team meeting | 1 hour | Medium                                               │
│  Tuesday | 13:00 - 13:30 | Break | 30 minutes | -                                                               │
│  Tuesday | 13:30 - 15:30 | Task 3: Respond to emails | 2 hours | Low                                            │
│                                                                                                                 │
│  Wednesday | 09:00 - 10:00 | Break and rest | 1 hour | -                                                        │
│  Wednesday | 10:00 - 12:00 | Task 1: Complete project report | 2 hours | High                                   │
│  Wednesday | 12:00 - 13:00 | Break | 1 hour | -                                                                 │
│  Wednesday | 13:00 - 14:00 | Task 2: Team meeting | 1 hour | Medium                                             │
│  Wednesday | 14:00 - 14:30 | Break | 30 minutes | -                                                             │
│  Wednesday | 14:30 - 16:30 | Task 3: Respond to emails | 2 hours | Low                                          │
│                                                                                                                 │
│  Thursday | 09:00 - 11:00 | Task 1: Complete project report | 2 hours | High                                    │
│  Thursday | 11:00 - 12:00 | Break | 1 hour | -                                                                  │
│  Thursday | 12:00 - 13:00 | Task 2: Team meeting | 1 hour | Medium                                              │
│  Thursday | 13:00 - 13:30 | Break | 30 minutes | -                                                              │
│  Thursday | 13:30 - 15:30 | Task 3: Respond to emails | 2 hours | Low                                           │
│                                                                                                                 │
│  Friday | 09:00 - 10:00 | Break and rest | 1 hour | -                                                           │
│  Friday | 10:00 - 12:00 | Task 1: Complete project report | 2 hours | High                                      │
│  Friday | 12:00 - 13:00 | Break | 1 hour | -           

In [25]:

# # Save result to PDF

# summary_text = result.raw   # string output

# pdf = FPDF()
# pdf.add_page()
# pdf.set_font("Arial", size=12)

# # Split summary text into lines
# for line in summary_text.split("\n"):
#     pdf.multi_cell(0, 10, line)

# # Save to output folder
# output_file_path = os.path.join("output", "summary.pdf")
# os.makedirs("output", exist_ok=True)
# pdf.output(output_file_path)


# print(f"Summary saved at {os.path.abspath(output_file_path)}")

In [26]:
# def save_pdf(text, filename):
#     pdf = FPDF()
#     pdf.add_page()
#     pdf.set_font("Arial", size=12)
#     for line in text.split("\n"):
#         pdf.multi_cell(0, 10, line)
#     os.makedirs("output", exist_ok=True)
#     pdf.output(os.path.join("output", filename))


In [27]:
def save_output(text, filename, table=False):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=11)

    if table:
        lines = [line.strip() for line in text.split("\n") if line.strip() and "|" in line]

        # Define columns
        headers = ["Day", "Time Slot", "Task", "Duration", "Priority"]
        col_widths = [30, 40, 70, 25, 25]

        # Header row
        pdf.set_fill_color(200, 200, 200)
        pdf.set_font("Arial", "B", 11)
        for i, header in enumerate(headers):
            pdf.cell(col_widths[i], 10, header, border=1, align="C", fill=True)
        pdf.ln()

        pdf.set_font("Arial", size=10)

        # Table rows
        for line in lines:
            parts = [p.strip() for p in line.split("|")]
            if len(parts) == 5:
                for i, val in enumerate(parts):
                    pdf.cell(col_widths[i], 8, val, border=1, align="C")
                pdf.ln()
    else:
        for line in text.split("\n"):
            pdf.multi_cell(0, 10, line)

    os.makedirs("output", exist_ok=True)
    output_path = os.path.join("output", filename)
    pdf.output(output_path)

In [28]:
save_output(result.tasks_output[0].raw, "summary.pdf")  # normal
save_output(result.tasks_output[1].raw, "timetable.pdf", table=True)  # table mode